<a href="https://colab.research.google.com/github/higherbar-ai/ai-workflows/blob/main/src/example-doc-conversion.ipynb" target="_parent"><img alt="Open In Colab" src="https://colab.research.google.com/assets/colab-badge.svg"/></a>

# About this notebook

This notebook provides a simple example of how to use the `ai-workflows` package to convert documents to Markdown format.

## Configuration

This notebook requires different settings depending on which AI service providers you want to use. If you're running in Google Colab, you configure these settings as "secrets"; just click the key icon in the left sidebar (and, once you create a secret, be sure to click the toggle to give the notebook access to the secret). If you're running this notebook in a different environment, you can set these settings in a `.env` file; the first time you run, it will write out a template `.env` file for you to fill in and direct you to its location. 

Following are the settings, regardless of the environment.

### OpenAI (direct)

To use OpenAI directly:

* `openai_api_key` - your OpenAI API key (get one from [the OpenAI API key page](https://platform.openai.com/api-keys), and be sure to fund your platform account with at least $5 to allow GPT-4o model access)
* `openai_model` (optional) - the model to use (defaults to `gpt-4o`)

### OpenAI (via Microsoft Azure)

To use OpenAI via Microsoft Azure:

* `azure_api_key` - your Azure API key
* `azure_api_base` - the base URL for the Azure API
* `azure_api_engine` - the engine to use (a.k.a. the "deployment")
* `azure_api_version` - the API version to use

### Anthropic (direct)

To use Anthropic directly:

* `anthropic_api_key` - your Anthropic API key
* `anthropic_model` - the model to use

### LangSmith (for tracing)

Optionally, you can add [LangSmith tracing](https://langchain.com/langsmith):

* `langsmith_api_key` - your LangSmith API key

## Setting up the runtime environment

This next code block installs all necessary Python and system packages into the current environment.

**If you're running in Google Colab and it prompts you to restart the notebook in the middle of the installation steps, just click CANCEL.**

In [ ]:
# install Google Colab Support and ai_workflows package
%pip install colab-or-not py-ai-workflows

# download NLTK data
import nltk
nltk.download('punkt', force=True)

# set up our notebook environment (including LibreOffice)
from colab_or_not import NotebookBridge
notebook_env = NotebookBridge(
    system_packages=["libreoffice"],
    config_path="~/.hbai/ai-workflows.env",
    config_template={
        "openai_api_key": "",
        "openai_model": "",
        "azure_api_key": "",
        "azure_api_base": "",
        "azure_api_engine": "",
        "azure_api_version": "",
        "anthropic_api_key": "",
        "anthropic_model": "",
        "langsmith_api_key": "",
    }
)
notebook_env.setup_environment()

## Initializing

The next code block initializes the notebook by loading settings and initializing the LLM interface.

In [ ]:
from ai_workflows.llm_utilities import LLMInterface
from ai_workflows.document_utilities import DocumentInterface

# read all supported settings
openai_api_key = notebook_env.get_setting('openai_api_key')
openai_model = notebook_env.get_setting('openai_model', 'gpt-4o')
azure_api_key = notebook_env.get_setting('azure_api_key')
azure_api_base = notebook_env.get_setting('azure_api_base')
azure_api_engine = notebook_env.get_setting('azure_api_engine')
azure_api_version = notebook_env.get_setting('azure_api_version')
anthropic_api_key = notebook_env.get_setting("anthropic_api_key")
anthropic_model = notebook_env.get_setting("anthropic_model")
langsmith_api_key = notebook_env.get_setting('langsmith_api_key')

# complain if we don't have the bare minimum to run
if (not openai_api_key
        and not (azure_api_key and azure_api_base and azure_api_engine and azure_api_version)
        and not (anthropic_api_key and anthropic_model)):
    raise Exception('We need settings set for OpenAI access (direct or via Azure) or for Anthropic access (direct). See the instructions above for more details.')

# initialize LLM interface
llm = LLMInterface(openai_api_key=openai_api_key, openai_model=openai_model, azure_api_key=azure_api_key, azure_api_base=azure_api_base, azure_api_engine=azure_api_engine, azure_api_version=azure_api_version, temperature = 0.0, total_response_timeout_seconds=600, number_of_retries=2, seconds_between_retries=5, langsmith_api_key=langsmith_api_key, anthropic_api_key=anthropic_api_key, anthropic_model=anthropic_model)

# initialize our document processor
doc_interface = DocumentInterface(llm_interface=llm)

# report success
print("Initialization successful.")

## Document selection and conversion

This next code block prompts you to upload or select one or more files to convert to markdown, then it converts them to Markdown.

### Where to find output files

If you're running in Google Colab, output files are saved into the content folder. Find, view, and download them by clicking on the folder icon in the left sidebar.

If you're running elsewhere, output files are saved into an `ai-workflows` subdirectory created off of your user home directory.

In [ ]:
import os

# prompt for one or more files
files_to_convert = notebook_env.get_input_files("File(s) to convert to Markdown")

# output files to ~/ai-workflows directory if local, otherwise /content if Google Colab
if notebook_env.is_colab:
    output_path_prefix = "/content"
else:
    output_path_prefix = os.path.expanduser("~/ai-workflows")
    os.makedirs(output_path_prefix, exist_ok=True)

# process files one-by-one
for file_to_convert in files_to_convert:
    print(f'Processing {file_to_convert}...')
    
    # convert to markdown
    markdown = doc_interface.convert_to_markdown(file_to_convert)

    # write the markdown to the output directory
    output_path = os.path.join(output_path_prefix, os.path.splitext(os.path.basename(file_to_convert))[0] + '.md')
    with open(output_path, 'w') as f:
        f.write(markdown)

    print(f"Conversion complete. Markdown saved to {output_path}")